In [8]:
import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd


url = 'https://hotels.1001tur.ru/opinion/russia/'
# Заголовки для запроса
headers = {"User-Agent": "Mozilla/5.0", "Accept-Language": "ru-RU,ru;q=0.9,en;q=0.8"}

# Выполняем запрос к странице
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")
# print(soup)
# Преобразуем весь HTML-код страницы в строку
html_text = soup.prettify()

    
s = BeautifulSoup(html_text, "html.parser")
div = s.find_all('div', class_='hotel-review-block__content')

data = []
for el in div:
    data.append(el.text.replace("\n", ' ').replace("  ", ' '))

df = pd.DataFrame(data)
df

,0
0,Россия Анапа ...
1,Россия Санкт-Петербург ...
2,Россия Сочи ...
3,Россия Калуга ...
4,Россия Лоо ...
5,Россия Роза Хутор ...
6,Россия Карелия ...
7,Россия Кисловодск ...
8,Россия Роза Хутор ...
9,Россия Калуга ...


In [3]:
# URL страницы с отзывами
import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd

url = "https://travel.yandex.ru/hotels/moscow/gamma-izmailovo/?adults=1&checkinDate=2024-11-14&checkoutDate=2024-11-23&childrenAges=&searchPagePollingId=e2e6db29c5fd2766cd7a5ad5ff35b910-0-newsearch&seed=portal-hotels-search"

# Заголовки для запроса
headers = {"User-Agent": "Mozilla/5.0", "Accept-Language": "ru-RU,ru;q=0.9,en;q=0.8"}

# Выполняем запрос к странице
response = requests.get(url, headers=headers)

# Парсим страницу с помощью BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")
# print(soup)
# Преобразуем весь HTML-код страницы в строку
html_text = soup.prettify()
data_list = []
# print(html_text.split("textReviews")[1], len(html_text.split("textReviews")[1]), html_text.split("textReviews")[1].find('"text":'))
pattern = re.compile(r'\{.*?"text":.*?\}')
matches = pattern.findall(html_text.split("textReviews")[1])
for match in matches:
    data = match.split('{"status":"ACCEPTED"},')

    if len(data) < 2:
        continue
    # print(list(filter(lambda el: el.startswith('"snippet":') or  el.startswith('"text":'),data[1].split(','))))
    # print(data[1].split('","'))
    data = list(
        filter(
            lambda el: el.startswith('"snippet":') or el.startswith('text":'),
            data[1].split('","'),
        )
    )
    # print(data)
    try:
        snippet = data[0].split(":")[1].strip('"')
        text = data[1].split(":")[1].strip('"')
        #print(snippet, text)
        if snippet is not None:
            # Добавляем данные в список
            data_list.append(
                {
                    # "snippet": snippet,
                    "text": snippet + text,
                }
            )
    except IndexError:
        continue

# Создаем DataFrame из списка
df = pd.DataFrame(data_list)

# Выводим DataFrame
df


,text
0,"Классный отель, все понравилосьШикарный отель!..."
1,Шикарный отель! Отзывчивый персонал! Комфортны...
2,Очень уютный отель Хорошая гостиница за свои д...
3,"Хорошая гостиница за свои деньги, цена качеств..."
4,"Всё понравилась , классное место Очень хорошая..."
5,Очень хорошая гостиницаОтличный отель! Простор...
6,"Отличный отель! Просторный, чистый, есть все н..."
7,"Отличный отель, очень понравилось!Очень быстро..."
8,"Очень быстро и удобно и вежливые сотрудники, в..."
9,Замечательный отельВсё супер


In [9]:
url = "https://www.booking.com/hotel/gb/royal-national.ru.html"

# Заголовки для запроса
headers = {
    'User-Agent': 'Mozilla/5.0',
    'Accept-Language': 'ru-RU,ru;q=0.9,en;q=0.8'
}

# Выполняем запрос к странице
response = requests.get(url, headers=headers)

# Парсим страницу с помощью BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')
# Преобразуем весь HTML-код страницы в строку
html_text = soup.prettify()

# Ищем все вхождения JSON-объектов с нужными полями
pattern = re.compile(r'\{.*?"averageScore":.*?\}')
matches = pattern.findall(html_text)

# Создаем список для хранения данных
data_list = []

for match in matches:
    try:

        # Очистка строки от лишних символов
        clean_match = match.replace('&quot;', '"').replace('\\n', '').replace('\\', '')

        # Загрузка строки как JSON
        data = json.loads(clean_match)
        average_score = data.get('averageScore', None)
        positive_text = data.get('positiveText', '')
        negative_text = data.get('negativeText', '')

        if average_score is not None:
        # Добавляем данные в список
            data_list.append({
            'averageScore': average_score,
            'positiveText': positive_text,
            'negativeText': negative_text
           })
    except json.JSONDecodeError:
        continue

# Создаем DataFrame из списка
df = pd.DataFrame(data_list)

# Выводим DataFrame
df



,averageScore,positiveText,negativeText
0,9,"Отель очень большой, много проживающих и поэто...","Нет нареканий, только привет Мохамеду с рецепш..."
1,10,"Отель находится в удобном месте,200 м от метр...",
2,9,"Удобное расположение: ~20 мин от Кингс кросс, ...",В биде не работала горячая вода. В принципе ра...
3,10,"Удобства, отличное цена и локация а так же про...","1.Отсутствие тапочек, номер предназначен для 4..."
4,9,Гостиница находится в центре. Это очень удобно...,Не удобные прдушки. Очень не хватало холодильн...
5,8,That the room was clean and tidy. We were faci...,The rooms having guests on either side were no...
6,10,"The silent , the beds, the pillows, the clean ...",Sometimes it was impossible to comunicate with...
7,10,Good location for what we needed. Check in eas...,Nothing
